# NWB Single Neurons Example

This example introduces using the NWB file format for spike related data. 

Adapted from: https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html

In [1]:
%config Completer.use_jedi = False

In [2]:
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np

# Import pynwb utilities
from pynwb import NWBFile, NWBHDF5IO
from pynwb.file import Subject
from pynwb.image import ImageSeries
from pynwb.behavior import Position
from pynwb.ecephys import ElectricalSeries, SpikeEventSeries

## Settings

In [3]:
# Set path for data files
data_path = 'data'

In [4]:
# Set random seed for consistency simulating data
np.random.seed(1234)

## Define Subject

The `Subject` object takes the following information: 
- species
- age
- sex
- description
- subject_id

The description can includes information such as the epilepsy diagnosis. 

In [5]:
# Set subject information
species = 'elf'
age = 1000
sex = 'Male'
description = 'A magical being.'
subject_id = '001'

In [6]:
# Create subject object
subject = Subject(age=str(age),
                  sex=sex,
                  description=description, 
                  species=species,
                  subject_id=subject_id)

## Initialize NWB File

The NWBFile is initialized with the following session information:
- identifier
- experimenter
- lab
- institution
- session_id
- session_description
- session_start_time

In [7]:
# Define information for the NWB file
session_desc = 'my first synthetic recording'
identifier = 'EXAMPLE_ID'
session_start_time = datetime.now(tzlocal())
experimenter = 'Dr. Bilbo Baggins'
lab = 'Bag End Laboratory'
institution = 'University of Middle Earth at the Shire'
experiment_desc = 'I went on an adventure with thirteen dwarves to reclaim vast treasures.'
session_id = 'LONELYMTN'

In [8]:
# Initialize an NWB file with some metadata information
nwbfile = NWBFile(session_desc, identifier, session_start_time,
                  subject=subject, experimenter=experimenter,
                  lab=lab, institution=institution,
                  experiment_description=experiment_desc,
                  session_id=session_id)

## Recording Equipment

Metadata about the recording equipment is stored in the NWB file. 

This is done in hierarchical manner, in which:
- `Device` stores general information about the recording device
- `ElectrodeGroup` stores information about group(s) of electrodes
- `Electrode` stores information about individual electrodes

#### Device

A Device is initialized with the following information:
- name
- description
- manufacturer

In [9]:
# Define information for a device
device_name = 'electrode_recorder'
device_desc = 'some kind of elven invention'
device_manu = 'Orc industries.'

In [10]:
# Create a device
device = nwbfile.create_device(name=device_name, description=device_desc, manufacturer=device_manu)

#### Electrode Group

An electrode group is defined with the following information:
- name
- description
- location

In [11]:
# Define information for an electrode group
electrode_name = 'tetrode1'
description = "an example tetrode"
location = "somewhere in the hippocampus"

In [12]:
# Create an electrode group
electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                 description=description,
                                                 location=location,
                                                 device=device)

#### Electrodes

Electrodes are defined with the following information:
- id
- position (x, y, z coordinates)
- impedence
- location
- filtering
- group

In [13]:
# Define information for electrodes
elec_ids = [1, 2, 3, 4]
epos = {'x' : 1.0, 'y' : 2.0, 'z' : 3.0}
impedence = np.inf
filtering = 'none'
location = 'CA1'

In [14]:
# Add a collection of electrodes
for idx in elec_ids:
    nwbfile.add_electrode(id=idx,
                          x=epos['x'], y=epos['y'], z=epos['z'],
                          imp=impedence,
                          location=location, 
                          filtering=filtering,
                          group=electrode_group)

## Electrode Table

In [15]:
# Grab a sub-selection of electrodes
electrode_table_region = nwbfile.create_electrode_table_region([0, 2], 'the first and third electrodes')

In [16]:
# Check the electrode table
electrode_table_region

electrodes hdmf.common.table.DynamicTableRegion at 0x140373255812240
    Target table: electrodes hdmf.common.table.DynamicTable at 0x140373255813056

## Add time series data

NWB files can also store field data, as time series. 

Time series data is stored in an `ElectricalSeries`, which takes:
- name
- data
- timestamps or start_time & rate
- resolution
- description
- comments

In [17]:
# Set meta data
rate = 10.0

# Simulate some example data
data_len = 1000
ephys_data = np.random.rand(data_len * 2).reshape((data_len, 2))
ephys_timestamps = np.arange(data_len) / rate

In [18]:
# Create a time series of electrophysiology data
ephys_ts = ElectricalSeries('test_ephys_data',
                            ephys_data,
                            electrode_table_region,
                            timestamps=ephys_timestamps,
                            # Alternatively, could specify starting_time and rate as follows
                            # starting_time=ephys_timestamps[0],
                            # rate=rate,
                            resolution=0.001,
                            comments="This data was randomly generated with numpy, using 1234 as the seed",
                            description="Random numbers generated with numpy.random.rand")

# Add the ephys acquisition to the NWB file
nwbfile.add_acquisition(ephys_ts)

## Add units & spike events



In [19]:
# Simulate some example 
spike_times = np.sort(np.random.randint(0, 100, 25)).astype('float')
waveform = np.ones(100)

In [20]:
# Add some units 
nwbfile.add_unit(spike_times, waveform_mean=waveform, id=1, electrodes=[0])
nwbfile.add_unit(spike_times, waveform_mean=waveform, id=2, electrodes=[0])

## Position Information

Spatial information such as a subjects position, can be stored in a `Position` object.

The `Position` object takes a name, and stores a spatial series, which takes:
- name
- data
- rate
- reference_frame
- description

In [21]:
# Initialize a position object for storing position data
position = Position(name='position')

In [22]:
# Create a spatial series of information in the position object
position.create_spatial_series(name = 'position1',
                               data = np.linspace(0, 1, 20),
                               rate = 50.,
                               reference_frame='start point')

position1 pynwb.behavior.SpatialSeries at 0x140373255850832
Fields:
  comments: no comments
  conversion: 1.0
  data: [0.         0.05263158 0.10526316 0.15789474 0.21052632 0.26315789
 0.31578947 0.36842105 0.42105263 0.47368421 0.52631579 0.57894737
 0.63157895 0.68421053 0.73684211 0.78947368 0.84210526 0.89473684
 0.94736842 1.        ]
  description: no description
  rate: 50.0
  reference_frame: start point
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: meters

In [23]:
# Check the position object
position

position pynwb.behavior.Position at 0x140373255813392
Fields:
  spatial_series: {
    position1 <class 'pynwb.behavior.SpatialSeries'>
  }

In [24]:
# Add the position data the the NWB file
nwbfile.add_acquisition(position)

## Event Information

Event information can also be added to the NWB file.

For example, an `epoch` can be added, defining a start and stop time, with associated tags. 

In [25]:
# Add an epoch marker to the NWB file
nwbfile.add_epoch(start_time=0., stop_time=10., tags=['Experiment Block'])

Trials can also similarly be added to the NWB file. 

In order to define events of interest, the `add_trial_column` method can be used to define fields of interest. 

After event fields are defined, trials can be added to the event, defining values for the start and end of the trial, as well as for any defined fields of interest. 

In [26]:
# Add a field to the event trial structure for the response time
nwbfile.add_trial_column('response_time', 'Time when response is given.')

In [27]:
# Add some trials to the NWB file
nwbfile.add_trial(start_time=1.5, response_time=2.25, stop_time=3.0)
nwbfile.add_trial(start_time=3.5, response_time=4.75, stop_time=5.0)

### Stimuli

Stimulu can also be added to the NWB file. 

For example, we can add a series of images to the file. 

In [28]:
# Create dummy images, as simulated stimuli
img = np.ones(shape=[3, 3, 3])
stimuli = ImageSeries('test', [img, img], rate=1., unit='RGB')

In [29]:
# Add stimuli to the NWB file
nwbfile.add_stimulus(stimuli)

## Check File

In [30]:
# Check out the details of the example NWB file
nwbfile

root pynwb.file.NWBFile at 0x140373220851776
Fields:
  acquisition: {
    position <class 'pynwb.behavior.Position'>,
    test_ephys_data <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    electrode_recorder <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    tetrode1 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  epoch_tags: {
    Experiment Block
  }
  epochs: epochs <class 'pynwb.epoch.TimeIntervals'>
  experiment_description: I went on an adventure with thirteen dwarves to reclaim vast treasures.
  experimenter: ['Dr. Bilbo Baggins']
  file_create_date: [datetime.datetime(2021, 5, 10, 0, 4, 26, 725809, tzinfo=tzlocal())]
  identifier: EXAMPLE_ID
  institution: University of Middle Earth at the Shire
  lab: Bag End Laboratory
  session_description: my first synthetic recording
  session_id: LONELYMTN
  session_start_time: 2021-05-10 00:04:26.718356-04:00
  stimulus: {
    test <class 'pynwb.image.Imag

### Save out test file

In [31]:
# Save out an example NWB file
with NWBHDF5IO(data_path + '/nwb_single_neurons.nwb', 'w') as io:
    io.write(nwbfile)